# Asymmetric Encryption

## RSA

In [15]:
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import rsa, padding

### Generating Keys

In [16]:
private_key = rsa.generate_private_key(
    public_exponent=65537,
    key_size=2048,
)

public_key = private_key.public_key()

### Encrypting

In [38]:
message = b"Hello World!"

message_encrypted = public_key.encrypt(
    message,
    padding.OAEP(
        mgf=padding.MGF1(algorithm=hashes.SHA256()),
        algorithm=hashes.SHA256(),
        label=None
    )
)

print(f"Encrypted Text: {message_encrypted.hex()}")

Encrypted Text: 4ff0ef97f2350a89d83d25d77fee64af5b3a154536a6e70e34802a9a140e45c388693cba9417dbca73c38a5dc4a840dc5d7f184b93f6986f4a28e64bfb6e5c772eec1037a8e78381cedc85c119ff08c20632537fd31779b1fee129ab0466399b8b3a5d768580e3af9eeb75cbe3f1277e9d02a75186c7991b904346e8926501f7bbc3a9743c90ae82a4d9b88ccbfd258adce4a641219b45681479e50b4c5fff24db480faed2647587e319ae8adc629a8ae3069608a61556f7c1b2ad6147b2cd8d5de2d9858399b145df0ed6948d4783f8d3f6be56adf387d99ce2d2d0b8aa1fee08b50a533d9e077d37f09d7f379fe80ebf88271afcfbb579c9a7fc409a7a7616


### Decrypting

In [40]:
message_decrypted = private_key.decrypt(
    message_encrypted,
    padding.OAEP(
        mgf=padding.MGF1(algorithm=hashes.SHA256()),
        algorithm=hashes.SHA256(),
        label=None
    )
)

print(f"Decrypted Message: {message_decrypted}")

Decrypted Message: b'Hello World!'


## Using PEM Files for Keys

In [41]:
from pathlib import Path

from cryptography.hazmat.primitives import serialization

### Storing the Keys

In [49]:
password = b"my secret"

encoding = serialization.Encoding.PEM
format_private = serialization.PrivateFormat.PKCS8
algorithm = serialization.BestAvailableEncryption(password)

key_pem = private_key.private_bytes(
   encoding=encoding,
   format=format_private,
   encryption_algorithm=algorithm,
)

public_key = private_key.public_key()

format_public = serialization.PublicFormat.SubjectPublicKeyInfo

public_pem = public_key.public_bytes(
   encoding=encoding,
   format=format_public,
)

key_pem_path = Path("key.pem")
key_pem_path.write_bytes(key_pem)

public_pem_path = Path("public.pem")
public_pem_path.write_bytes(public_pem)

451

### Loading the Keys

#### Wrong Password

In [56]:
private_pem_bytes = Path("key.pem").read_bytes()
public_pem_bytes = Path("public.pem").read_bytes()

guess_password = b"my pass"

try:
    private_key = serialization.load_pem_private_key(
        private_pem_text,
        password=guess_password,
    )
    public_key = serialization.load_pem_public_key(public_pem_bytes)
except ValueError:
    print("Incorrect Password")

Incorrect Password


#### Right Password

In [57]:
private_pem_text = Path("key.pem").read_bytes()
public_pem_bytes = Path("public.pem").read_bytes()

try:
    private_key = serialization.load_pem_private_key(
        private_pem_text,
        password=password,
    )
    public_key = serialization.load_pem_public_key(public_pem_bytes)
except ValueError:
    print("Incorrect Password")

### Testing Keys

#### Encrypting

In [58]:
message = b"Hello World!"

message_encrypted = public_key.encrypt(
    message,
    padding.OAEP(
        mgf=padding.MGF1(algorithm=hashes.SHA256()),
        algorithm=hashes.SHA256(),
        label=None
    )
)

print(f"Encrypted Text: {message_encrypted.hex()}")

Encrypted Text: 9b00e74a22ea4597bfa37cd53c575ad18a0c49867957de5b9dc10ec7e99545a3e90ef39e3481553581a853387287eca71de53b32cd6106f70bacf4311b23117cc46d5d0f949988278d8cdc22418fe8b5fbf704d66466f155fbdb3b88fb516278e35df663a28ef8eaa25c41c6a5e4954bbe32682fa761ad553b2a55ecda82b52d76ff56cb2c4daa6570539e57a6719ac6710ba9b6b4f66f90f75eb3088598e52b9f94805748ea5277dd3cf61903f160f638a126d944ebc0fbb49522428ca463f41b3839f9e762f02aceaa2c6b388f786de2f9834516abf38076f81154e566fa982217ad741b3929be812f2e6f108bc85d8e789e452421bcc0e45020e701bef020


#### Decrypting

In [59]:
message_decrypted = private_key.decrypt(
    message_encrypted,
    padding.OAEP(
        mgf=padding.MGF1(algorithm=hashes.SHA256()),
        algorithm=hashes.SHA256(),
        label=None
    )
)

print(f"Decrypted Message: {message_decrypted}")

Decrypted Message: b'Hello World!'
